# Unsloth를 활용한 LLM 파인튜닝

이 노트북에서는 Unsloth 라이브러리를 사용하여 PDF Q&A 데이터로 LLM을 효율적으로 파인튜닝하는 방법을 설명합니다.

## 1. 환경 설정 및 라이브러리 체크

In [ ]:
# Setup script for Triton runtime with CUDA support
# This script checks if the Triton runtime is correctly set up with CUDA support.
# 필수 라이브러리 임포트 및 버전 확인

import torch, triton, flash_attn, trl
print(f"PyTorch 버전: {torch.__version__}")      # PyTorch 2.2.2 (CUDA 12.1)
print(f"Triton 버전: {triton.__version__}")      # 2.2.0
print(f"Flash-Attention 버전: {flash_attn.__version__}")
print(f"TRL 버전: {trl.__version__}") 

# Triton 기능 테스트
from triton.runtime.jit import get_cuda_stream     # 오류 없으면 정상

## 2. 모델 로드 및 설정

- `Unsloth`는 Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes 등을 지원합니다. 그리고 Yi, Qwen([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, 모든 Llama, Mistral 파생 아키텍처도 지원합니다.

- 사용방법 및 더 많은 양자화 모델 카탈로그 참조: https://huggingface.co/unsloth

Unsloth는 Llama, Mistral, Gemma 등 다양한 모델을 지원하며, 다양한 양자화와 LoRA를 통해 메모리 사용량을 크게 줄이고 훈련 속도를 높입니다.

### `FastLanguageModel.from_pretrained` 모델 로딩 가이드

### 핵심 파라미터
- **모델명(`model_name`)**: 사전 훈련된 모델 지정
- **최대 시퀀스 길이(`max_seq_length`)**: 처리할 입력 데이터의 최대 길이
- **데이터 타입(`dtype`)**: 자동 감지 또는 `Float16`/`Bfloat16` 지정 가능
- **양자화(`True`)**: 메모리 사용량 감소를 위한 선택적 옵션

### 최적화 옵션
- 사전 정의된 4비트 모델 목록(`fourbit_models`)을 활용하면 다운로드 시간 단축 및 메모리 효율성 향상
- 특정 게이트 모델 사용 시 `token` 파라미터로 액세스 권한 제공 가능


In [ ]:
from unsloth import FastLanguageModel
import torch

# 기본 설정
max_seq_length = 4096  # 최대 시퀀스 길이
dtype = None  # 자동 데이터 타입 감지

# 고속 다운로드 지원 4비트 사전 양자화 모델 목록
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # TTS 모델 지원
]  

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-14B",  # 모델명
    max_seq_length=max_seq_length,  # 시퀀스 길이
    dtype=dtype,  # 데이터 타입
    load_in_4bit=True,  # 4비트 양자화
    load_in_8bit=False,  # 8비트 양자화
    full_finetuning = False,  # 전체 파인튜닝 비활성화
    #use_flash_attention=True,  # 플래시 어텐션
    # token = "hf_...",  # 게이트 모델용 토큰
)

## 3. LoRA 어댑터 구성

LoRA 어댑터를 추가하여 모든 파라미터 중 단 1% ~ 10%의 파라미터만 업데이트

FastLanguageModel을 사용하여 특정 모듈에 대한 성능 향상 기법을 적용한 모델을 구성합니다.

- `FastLanguageModel.get_peft_model` 함수를 호출하여 모델을 초기화하고, 성능 향상을 위한 여러 파라미터를 설정합니다.
- `r` 파라미터를 통해 성능 향상 기법의 강도를 조절합니다. 권장 값으로는 8, 16, 32, 64, 128 등이 있습니다.
- `target_modules` 리스트에는 성능 향상을 적용할 모델의 모듈 이름들이 포함됩니다.
- `lora_alpha`와 `lora_dropout`을 설정하여 LoRA(Low-Rank Adaptation) 기법의 세부 파라미터를 조정합니다.
- `bias` 옵션을 통해 모델의 바이어스 사용 여부를 설정할 수 있으며, 최적화를 위해 "none"으로 설정하는 것이 권장됩니다.
- `use_gradient_checkpointing` 옵션을 "unsloth"로 설정하여 VRAM 사용량을 줄이고, 더 큰 배치 크기로 학습할 수 있도록 합니다.
- `use_rslora` 옵션을 통해 Rank Stabilized LoRA를 사용할지 여부를 결정합니다.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=32,  # Best to choose alpha = rank or rank*2
    lora_dropout=0.05,  # 드롭아웃을 지원합니다. Supports any, but = 0 is optimized
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.  but = "none" is optimized
    
    use_gradient_checkpointing="unsloth",# 메모리 절약을 위해 그래디언트 체크포인트를 사용합니다. True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다. 
    random_state=123,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

## 4. 학습 데이터 준비

모델 학습을 위해 QA 데이터셋을 적절한 형식으로 준비합니다.

- **데이터셋 로드**: `load_dataset` 함수를 통해 Q&A 학습용 데이터셋의 "train" 분할 불러오기
- **포맷 변환**: 각 예제를 모델 학습에 적합한 형식으로 변환

### 포맷팅 프로세스
1. `formatting_prompts_func` 함수가 "instruction"과 "output" 필드를 처리
2. 데이터를 알파카(Alpaca) 형식으로 구조화
3. 각 항목 끝에 `EOS_TOKEN` 추가하여 생성 종료 지점 명시

#### 결과 형식
```python
{
    "text": f"### Instruction:\n{instruction}\n\n### Response:\n{output}{EOS_TOKEN}"
}
```

이 전처리 과정을 통해 AI 모델이 효과적으로 학습할 수 있는 표준화된 데이터 형식이 생성됩니다.

**[중요]**

- llama모델의 경우, 토큰화된 출력에 **EOS_TOKEN**을 추가하는 것을 잊지 마세요! 그렇지 않으면 무한 생성이 발생할 수 있습니다.

**[참고]**

- 오직 완성된 텍스트만을 학습하고자 한다면, TRL의 문서를 [여기](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)에서 확인


In [ ]:
from datasets import load_dataset

# 종료 토큰 설정 (무한 생성 방지를 위해 필요)
EOS_TOKEN = tokenizer.eos_token

# 프롬프트 템플릿 정의
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# 데이터 포매팅 함수
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    
    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN #여기 추가함.
        texts.append(text)
    
    return {"text": texts}

In [ ]:
# JSONL 파일에서 데이터셋 로드 및 포매팅
dataset = load_dataset('json', data_files='data/train_data.jsonl', split='train')

dataset = dataset.map(
    formatting_prompts_func, 
    batched=True)

# 데이터셋 확인
print(dataset)

## 5. 모델 훈련

SFTTrainer를 사용하여 모델을 효율적으로 파인튜닝합니다.

- SFTTrainer 참고 문서: [TRL SFT 문서 클릭](https://huggingface.co/docs/trl/sft_trainer)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling

# DataCollator 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # MLM 비활성화
)

tokenizer.padding_side = "right"  # 패딩 방향 설정

# 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 훈련 모델
    tokenizer=tokenizer,  # 사용 토크나이저
    train_dataset=dataset,  # 훈련 데이터
    eval_dataset=dataset,  # 평가 데이터
    dataset_text_field="text",  # 텍스트 필드명
    max_seq_length=max_seq_length,  # 최대 길이
    dataset_num_proc=2,  # 프로세스 수
    packing=False,  # 시퀀스 패킹 비활성화
    data_collator=data_collator,  # 필수 데이터 수집기
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 디바이스당 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적
        warmup_steps=5,  # 웜업 단계
        num_train_epochs=3,  # 총 에폭
        max_steps=100,  # 최대 단계
        do_eval=True,  # 평가 활성화
        eval_strategy="steps",  # 평가 전략
        logging_steps=1,  # 로깅 간격
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 조건부 활성화
        bf16=torch.cuda.is_bf16_supported(),  # bf16 조건부 활성화
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소율
        lr_scheduler_type="cosine",  # 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 결과 저장 경로
    ),
)

- GPU의 현재 메모리 상태를 확인합니다.
- `torch.cuda.get_device_properties(0)`를 사용하여 첫 번째 GPU의 속성을 조회합니다.
- `torch.cuda.max_memory_reserved()`를 통해 현재 예약된 최대 메모리를 GB 단위로 계산합니다.

In [ ]:
# GPU 메모리 상태 확인
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 최대 메모리 = {max_memory} GB.")
print(f"현재 예약된 메모리 = {start_gpu_memory} GB.")

#### PyTorch 2.3 미만 버전 패치 적용
- unsloth_zoo는 PyTorch ≥ 2.3에서 새롭게 추가된 torch.amp.is_autocast_available() API를 호출합니다.  
- monkey patch 패치 적용

In [ ]:
# SageMaker 노트북의 PyTorch 버전이 2.2.2이기 때문에, torch.amp.is_autocast_available() 함수가 없어 AttributeError가 발생
# monkey patch 패치 적용
# PyTorch 2.3 미만 버전 패치 적용
import torch.amp
if not hasattr(torch.amp, 'is_autocast_available'):
    def is_autocast_available(device_type):
        if device_type == 'cuda':
            return True
        elif device_type == 'cpu':
            return hasattr(torch.cpu, 'amp') and hasattr(torch.cpu.amp, 'autocast')
        else:
            return False
    torch.amp.is_autocast_available = is_autocast_available
    

### 모델 훈련 실행

In [ ]:
trainer_stats = trainer.train() 

## 6. 훈련 결과 및 메모리 사용량 분석

In [ ]:
# 메모리 사용량 및 훈련 시간 통계 계산
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# 통계 출력
print(f"훈련 시간: {trainer_stats.metrics['train_runtime']} 초 ({round(trainer_stats.metrics['train_runtime']/60, 2)} 분)")
print(f"최대 사용 메모리: {used_memory} GB (전체의 {used_percentage}%)")
print(f"LoRA 학습에 사용된 메모리: {used_memory_for_lora} GB (전체의 {lora_percentage}%)")

## 7. 모델 추론 및 테스트

파인튜닝된 모델을 사용하여 질문에 답변을 생성합니다.

In [ ]:
from transformers import TextStreamer, StoppingCriteria, StoppingCriteriaList

# 생성 중단 조건 설정
class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_token_id in input_ids[0] #$ 입력된 ID 중 정지 토큰 ID가 있으면 True를 반환합니다.

stop_token = "<|end_of_text|>" # end_token으로 사용할 토큰을 설정, 모델에 따라 다를 수 있습니다. llama-3 계열은 <|end_of_text|>를 Qwen3 계열은 <|im_end|>를 사용합니다.

stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[0]
stopping_criteria = StoppingCriteriaList([StopOnToken(stop_token_id)]) #    정지 조건을 설정합니다.

#### 추론 모드 설정 (2배 빠른 생성 속도)

In [ ]:
FastLanguageModel.for_inference(model)

#### 추론 예시 1: 학습 테스트 더미 확인용 인물 정보 질문

In [ ]:
# 텍스트 스트리밍 설정
text_streamer = TextStreamer(tokenizer)

# 질문 입력 및 응답 생성
inputs = tokenizer(
    [prompt.format("전현상은 누구입니까?", "")],
    return_tensors="pt"
).to("cuda")

# 생성 실행
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,
    stopping_criteria=stopping_criteria
)

#### 추론 예시 2: 주소 정보 질문

In [ ]:
# 질문 입력 및 응답 생성
inputs = tokenizer(
    [prompt.format("금융보안교육센터의 주소?", "")],
    return_tensors="pt"
).to("cuda")

# 생성 실행
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,
    stopping_criteria=stopping_criteria
)

(예시2)